In [3]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup


In [4]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.exc import IntegrityError

In [5]:
import pandas as pd

In [6]:
def get_soup(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'html.parser')
    return soup

N1

In [7]:
top250 = get_soup('https://www.imdb.com/pt/chart/top/')
url_lista = top250.find('ul', class_='ipc-metadata-list')
top250_filmes = url_lista.find_all('h3', class_='ipc-title__text')
lista_filmes = []

for filme in top250_filmes:
    lista_filmes.append(filme.get_text())

for filme in lista_filmes[:10]:
    print(filme)

Um Sonho de Liberdade
O Poderoso Chefão
Batman: O Cavaleiro das Trevas
O Poderoso Chefão: Parte II
12 Homens e uma Sentença
O Senhor dos Anéis: O Retorno do Rei
A Lista de Schindler
O Senhor dos Anéis: A Sociedade do Anel
Pulp Fiction: Tempo de Violência
Três Homens em Conflito


N2

In [8]:
lista_organizada_filmes = []
li_lista = top250.find_all('li', class_='ipc-metadata-list-summary-item')

for li in li_lista:
    titulo = li.find('h3', class_='ipc-title__text').get_text()
    ano = li.find('span', class_='cli-title-metadata-item').get_text()
    nota_imbd = li.find('span', class_='ipc-rating-star--rating').get_text()

    lista_organizada_filmes.append({'titulo': titulo, 'ano': ano, 'nota_imbd': nota_imbd})


for filme in lista_organizada_filmes[:5]:
    print(f"{filme['titulo']} ({filme['ano']}) - Nota IMDB: {filme['nota_imbd']}")


Um Sonho de Liberdade (1994) - Nota IMDB: 9,3
O Poderoso Chefão (1972) - Nota IMDB: 9,2
Batman: O Cavaleiro das Trevas (2008) - Nota IMDB: 9,1
O Poderoso Chefão: Parte II (1974) - Nota IMDB: 9,0
12 Homens e uma Sentença (1957) - Nota IMDB: 9,0


N3

In [9]:
class TV:
    def __init__(self, title, year):
        self.title = title
        self.year = year
    
    def __str__(self):
        return f"{self.title} ({self.year})"     

N4

In [10]:
class Movie(TV):
    def __init__(self, title, year, rating):
        super().__init__(title, year)
        self.rating = rating
    
    def __str__(self):
        return f"{super().__str__()} - Nota: {self.rating}"
    
class Series(TV):
    def __init__(self, title, year, seasons, episodes):
        super().__init__(title, year)
        self.seasons = seasons
        self.episodes = episodes
    
    def __str__(self):
        return f"{super().__str__()} - Temporadas: {self.seasons} Episódios: {self.episodes}"

N5

In [12]:
catalog = []

for dados in lista_organizada_filmes:
    ano_int = int(dados['ano'])
    nota_float = float(dados['nota_imbd'].replace(',', '.'))
    filme = Movie(dados['titulo'], ano_int, nota_float)
    catalog.append(filme)

serie1 = Series("Supernatural", 2005, 15, 327)
serie2 = Series("Dexter", 2006, 8, 96)

catalog.append(serie1)
catalog.append(serie2)

for item in catalog:
    print(item)

Um Sonho de Liberdade (1994) - Nota: 9.3
O Poderoso Chefão (1972) - Nota: 9.2
Batman: O Cavaleiro das Trevas (2008) - Nota: 9.1
O Poderoso Chefão: Parte II (1974) - Nota: 9.0
12 Homens e uma Sentença (1957) - Nota: 9.0
O Senhor dos Anéis: O Retorno do Rei (2003) - Nota: 9.0
A Lista de Schindler (1993) - Nota: 9.0
O Senhor dos Anéis: A Sociedade do Anel (2001) - Nota: 8.9
Pulp Fiction: Tempo de Violência (1994) - Nota: 8.8
Três Homens em Conflito (1966) - Nota: 8.8
O Senhor dos Anéis: As Duas Torres (2002) - Nota: 8.8
Forrest Gump: O Contador de Histórias (1994) - Nota: 8.8
Clube da Luta (1999) - Nota: 8.8
A Origem (2010) - Nota: 8.8
Star Wars: Episódio V - O Império Contra-Ataca (1980) - Nota: 8.7
Matrix (1999) - Nota: 8.7
Os Bons Companheiros (1990) - Nota: 8.7
Interestelar (2014) - Nota: 8.7
Um Estranho no Ninho (1975) - Nota: 8.6
Seven - Os Sete Crimes Capitais (1995) - Nota: 8.6
A Felicidade Não se Compra (1946) - Nota: 8.6
O Silêncio dos Inocentes (1991) - Nota: 8.6
Os Sete Samura

N6

In [10]:
engine = create_engine('sqlite:///imdb.db', echo=False)
Base = declarative_base()

class MovieBase(Base):
    __tablename__ = 'movies'
    
    id = Column(Integer, primary_key=True)
    title = Column(String, unique=True)
    year = Column(Integer)
    rating = Column(Float)
    
class SeriesBase(Base):
    __tablename__ = 'series'
    
    id = Column(Integer, primary_key=True)
    title = Column(String, unique=True)
    year = Column(Integer)
    seasons = Column(Integer)
    episodes = Column(Integer)

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

for item in catalog:
    objeto = None
    if isinstance(item, Movie):
       objeto = MovieBase(title=item.title, year=item.year, rating=item.rating) 
    elif isinstance(item, Series):
        objeto = SeriesBase(title=item.title, year=item.year, seasons=item.seasons, episodes=item.episodes) 

    if objeto:
        try:
            session.add(objeto)
            session.commit()
        except IntegrityError:
            session.rollback()
            print(f"Item '{objeto.title}' já existe no banco de dados.")
        except Exception as e:
            session.rollback()
            print(e)       

N7

In [13]:
try:
    engine = create_engine('sqlite:///imdb.db', echo=False)
    df_movies = pd.read_sql_table('movies', engine)
    df_series = pd.read_sql_table('series', engine)

    print(df_movies.head())
    print(df_series.head())
except Exception as e:
    print(e)

   id                           title  year  rating
0   1           Um Sonho de Liberdade  1994     9.3
1   2               O Poderoso Chefão  1972     9.2
2   3  Batman: O Cavaleiro das Trevas  2008     9.1
3   4     O Poderoso Chefão: Parte II  1974     9.0
4   5        12 Homens e uma Sentença  1957     9.0
   id         title  year  seasons  episodes
0   1  Supernatural  2005       15       327
1   2        Dexter  2006        8        96


N8

In [21]:
df_organizado = df_movies.sort_values(by='rating', ascending=False)
df_filtrado = df_organizado[df_organizado['rating'] >= 9.0] ## no enunciado esta maior que 9, mas coloquei maior ou igual para aparecer mais filmes

print(df_filtrado.head())

try:
    df_movies.to_csv('movies.csv', index=False, encoding='utf-8')
    df_series.to_csv('series.csv', index=False, encoding='utf-8')

    df_movies.to_json('movies.json', orient='records', indent=4, force_ascii=False)
    df_series.to_json('series.json', orient='records', indent=4, force_ascii=False)

except Exception as e:
    print(e)

   id                           title  year  rating
0   1           Um Sonho de Liberdade  1994     9.3
1   2               O Poderoso Chefão  1972     9.2
2   3  Batman: O Cavaleiro das Trevas  2008     9.1
3   4     O Poderoso Chefão: Parte II  1974     9.0
4   5        12 Homens e uma Sentença  1957     9.0


In [22]:
def classificar_filmes(nota):
    if nota >= 9.0:
        return "Obra-prima"
    elif nota >= 8.0:
        return "Excelente"
    elif nota >= 7.0:
        return "Bom"
    else:
        return "Mediano"
    
df_movies['categorias'] = df_movies['rating'].apply(classificar_filmes)
selecao = df_movies[['title', 'rating', 'categorias']]
print(selecao.head(10))

                                     title  rating  categorias
0                    Um Sonho de Liberdade     9.3  Obra-prima
1                        O Poderoso Chefão     9.2  Obra-prima
2           Batman: O Cavaleiro das Trevas     9.1  Obra-prima
3              O Poderoso Chefão: Parte II     9.0  Obra-prima
4                 12 Homens e uma Sentença     9.0  Obra-prima
5     O Senhor dos Anéis: O Retorno do Rei     9.0  Obra-prima
6                     A Lista de Schindler     9.0  Obra-prima
7  O Senhor dos Anéis: A Sociedade do Anel     8.9   Excelente
8         Pulp Fiction: Tempo de Violência     8.8   Excelente
9                  Três Homens em Conflito     8.8   Excelente


N10

In [26]:
tabela_resumo = pd.crosstab(df_movies['year'], df_movies['categorias'])
print(tabela_resumo)

categorias  Excelente  Obra-prima
year                             
1946                1           0
1954                1           0
1957                0           1
1966                1           0
1972                0           1
1974                0           1
1975                1           0
1980                1           0
1990                1           0
1991                1           0
1993                0           1
1994                2           1
1995                1           0
1998                1           0
1999                3           0
2001                1           0
2002                1           0
2003                0           1
2008                0           1
2010                1           0
2014                1           0
